# Asignación y Desenmascarado

Saquemos las máscaras a los tweets

In [122]:
import pandas as pd

df_titulares = pd.read_csv("../data/titulos_filtrados.csv")

df_titulares = df_titulares.rename(columns={"Unnamed: 0": "id"}).set_index("id")

df_titulares

,medio,fecha,titulo
id,,,
23,la izquierda diario,2019-09-21 00:00:00,Barrio Aeroclub: la Izquierda propone urbanización y adjudicación de terrenos
24,la izquierda diario,2019-09-21 00:00:00,Elecciones municipales: el #FITUnidad busca fortalecer la izquierda en la ciudad de Neuquén
31,la izquierda diario,2019-09-21 00:00:00,[Videos] Mirá todos los spots del Frente de Izquierda Unidad en Mendoza
65,la nación,2019-09-22 00:00:00,Tipo de cambio: por qué nos preocupa el dólar y qué efectos tienen los controles
75,la izquierda diario,2019-09-22 00:00:00,Brutalidad de la Policía de Urtubey en el recital de Sara Hebe en Salta
...,...,...,...
22608,prensa obrera,2019-10-27 23:05:46,Declaración del Partido Obrero-Frente de Izquierda Unidad ante los resultados de las elecciones 2019
22611,la nación,2019-10-27 23:16:00,"Resultados elecciones 2019: Alberto Fernández, tras el triunfo: ""Nos dijeron 'no vuelven más', pero volvimos y vamos a ser mejores""."
22612,la nación,2019-10-27 23:21:00,"Elecciones 2019. La llamada de Macri a Alberto: ""En política se gana y se pierde"""


In [130]:
[x for x in df["id"].values if x not in df_titulares.index]

[39]

In [133]:
df_titulares.sort_index()

,medio,fecha,titulo
id,,,
23,la izquierda diario,2019-09-21 00:00:00,Barrio Aeroclub: la Izquierda propone urbanización y adjudicación de terrenos
24,la izquierda diario,2019-09-21 00:00:00,Elecciones municipales: el #FITUnidad busca fortalecer la izquierda en la ciudad de Neuquén
31,la izquierda diario,2019-09-21 00:00:00,[Videos] Mirá todos los spots del Frente de Izquierda Unidad en Mendoza
65,la nación,2019-09-22 00:00:00,Tipo de cambio: por qué nos preocupa el dólar y qué efectos tienen los controles
75,la izquierda diario,2019-09-22 00:00:00,Brutalidad de la Policía de Urtubey en el recital de Sara Hebe en Salta
...,...,...,...
22608,prensa obrera,2019-10-27 23:05:46,Declaración del Partido Obrero-Frente de Izquierda Unidad ante los resultados de las elecciones 2019
22611,la nación,2019-10-27 23:16:00,"Resultados elecciones 2019: Alberto Fernández, tras el triunfo: ""Nos dijeron 'no vuelven más', pero volvimos y vamos a ser mejores""."
22612,la nación,2019-10-27 23:21:00,"Elecciones 2019. La llamada de Macri a Alberto: ""En política se gana y se pierde"""


In [144]:
import pandas as pd
import glob
import numpy as np
import re



paths = glob.glob("../data/etiquetados_final/*.xlsx")

dfs = []
annotators = []

for path in paths:
    name = os.path.basename(path).split(".")[0]
    
    df = pd.read_excel(path)
    df = df.iloc[3:]
    annotators.append(name)

    df.rename(columns={df.columns[0]: "id", "TITULO": "titulo_mask", " ": "etiquetado", "Etiquetado?": "etiquetado"}, inplace=True)

    # Por algún motivo perdimos este titular...
    df.set_index("id", inplace=True)

    ids = [x for x in df.index if x not in {39}]
    
    df.loc[ids, "titulo"] = df_titulares.loc[ids, "titulo"]
    df["anotador"] = name
    df.reset_index(inplace=True)
    dfs.append(df)
    
annotators = sorted(annotators)
annotator_df = pd.concat(dfs).set_index(["id", "anotador"])

## Asignación

Primero, veamos cómo asignarlo

In [64]:
ids = dfs[0]["id"]

label_cols = ['TARGET A', 'TARGET B', 'TARGET C', 'TARGET D',
       'TARGET E', 'TARGET F']

total = 0

accepted_ids = []

for _, row in dfs[0].iterrows():
    idx = row["id"]
    labels = annotator_df.loc[idx]

    assert len(labels) == 3
    annotated = labels[label_cols].notna()

    """
    Chequeamos si la anotación difirió
    """
    if not ( all(annotated.iloc[0] == annotated.iloc[1]) and all(annotated.iloc[1] == annotated.iloc[2])):
        print("="*80 + "\n" * 2)
        print(idx, row["titulo"])
        print(annotated.sum(axis=1))
        print(annotated)
        total+=1
    else:
        accepted_ids.append(idx)



695 Marco [TARGET B]  : votó en contra del aborto y disputa una banca con Myriam Bregman
anotador
e2     1
e3     2
et1    1
dtype: int64
          TARGET A  TARGET B  TARGET C  TARGET D  TARGET E  TARGET F
anotador                                                            
e2           False      True     False     False     False     False
e3           False      True      True     False     False     False
et1          False      True     False     False     False     False


1029 Bregman y [TARGET D]    en la movilización contra la crisis climática y ecológica
anotador
e2     2
e3     1
et1    1
dtype: int64
          TARGET A  TARGET B  TARGET C  TARGET D  TARGET E  TARGET F
anotador                                                            
e2           False     False     False      True      True     False
e3           False     False     False      True     False     False
et1          False     False     False      True     False     False


1289 Sergio Massa sobre el act

Ok, quedémonos con accepted_ids

Para cada titular, hagamos lo siguiente:

- Votación mayoritaria (elegimos al que tenga 2 votos)
- Si no ocurre...mmm, tiremos el titular mejor

In [102]:
problematic = []

dataset = []

for idx in accepted_ids:
    annotated_columns = [col for col, annotated in annotator_df.loc[idx, label_cols].notna().all().items() if annotated]

    for col in annotated_columns:
        counts = annotator_df.loc[idx, col].value_counts()

        if counts.max() == 1:
            problematic.append(idx)
        else:
            dataset.append
print(len(problematic))

114


In [114]:
pd.options.display.max_colwidth = 190
annotator_df.loc[problematic[-22]]

,titulo,etiquetado,TARGET A,TARGET B,TARGET C,TARGET D,TARGET E,TARGET F
anotador,,,,,,,,
e2,"Subordinarse a la elección de 2021, la tentación que [TARGET E] debe evitar",SI,NaN,NaN,NaN,NaN,NEU,NaN
e3,"Subordinarse a la elección de 2021, la tentación que [TARGET E] debe evitar",SI,NaN,NaN,NaN,NaN,POS,NaN
et1,"Subordinarse a la elección de 2021, la tentación que [TARGET E] debe evitar",SI,NaN,NaN,NaN,NaN,NEG,NaN


In [100]:

annotator_df.loc[idx, "TARGET C"].value_counts()

3

## Tamaño del dataset

In [77]:
import matplotlib.pyplot as plt
import seaborn as sns 


sarasa = annotator_df.loc[[(x, "et1") for x in accepted_ids]]

print("Cantidad de targets por título")
sarasa[label_cols].notna().sum(axis=1).value_counts().sort_index()


Cantidad de targets por título


0     160
1    1723
2     278
3      21
4       1
dtype: int64

Ok, hay muy poquitos con más de 1...

Haciendo la cuenta

In [81]:
pepe = sarasa[label_cols].notna().sum(axis=1).value_counts().sort_index()

len(accepted_ids), sum(p * q for p, q in pepe.items())

(2183, 2346)

Tenemos 2183 titulares, y 2346 targets